In [9]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import OPTICS
from sklearn.manifold import TSNE

from scipy.spatial.distance import pdist, squareform

import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [2]:
df_prices = pd.read_csv('stock_prices.csv',index_col='date',parse_dates=True)
df_prices = df_prices[df_prices.index < '2023-12-31']
df_prices

,ADN,AEE,AEP,AES,AGR,ALCE,ALE,AMPS,ARIS,ARTNA,...,TVE,UGI,UTL,VGAS,VIA,VST,WEC,WTRG,XEL,YORW
date,,,,,,,,,,,,,,,,,,,,,
2023-03-01,52.200001,77.833527,82.796310,23.525192,36.142548,10.200,58.352753,6.63,12.934539,53.065838,...,20.497381,34.530087,51.602707,10.200,29.049999,21.039583,82.554329,41.228195,60.526001,41.644726
2023-03-02,51.599998,79.110909,84.359062,23.534746,36.787613,10.205,58.724483,6.64,13.258378,53.962120,...,20.390877,34.753704,52.164341,9.080,28.400000,20.632555,83.937073,41.170086,61.549595,42.036217
2023-03-03,53.400002,80.417099,85.413658,23.974112,37.280895,10.210,59.334507,6.96,13.115508,53.923149,...,20.671663,35.033226,52.755028,8.990,27.350000,20.748846,85.434242,40.685844,62.276634,42.496220
2023-03-06,52.200001,80.532349,85.682106,23.878599,37.413712,10.220,58.915123,7.14,12.610699,53.747795,...,20.671663,34.576675,52.144970,9.000,25.850000,23.627132,85.672638,40.966709,62.228806,42.643028
2023-03-07,51.599998,79.178139,84.972641,23.410582,36.759155,10.230,57.799915,7.31,10.667661,53.708828,...,20.661982,34.353054,51.931942,8.896,24.750000,25.691357,84.747643,40.191921,60.851261,44.365585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22,6.354000,71.414375,79.465294,18.722160,31.887262,5.000,60.869202,6.64,8.305835,42.341656,...,21.622562,24.148851,51.537193,2.230,9.390000,37.889172,81.254288,36.819717,60.886734,38.234615
2023-12-26,6.099000,71.117096,79.811317,18.977016,32.133762,2.100,60.928459,6.88,8.454685,42.530109,...,21.671930,24.493694,52.221313,2.200,9.740000,38.068550,81.940056,37.097301,61.073143,39.065159
2023-12-27,6.972000,71.047737,79.890396,18.898602,31.946423,1.780,60.582783,6.92,8.474531,42.292065,...,21.849648,24.503548,52.578251,2.200,9.440000,38.247932,81.753914,37.077480,60.885166,39.114597


In [3]:
# Initializing the StandardScaler
scaler = StandardScaler()

# Fitting and transforming the data
df_scaled = scaler.fit_transform(df_prices)

# Converting the scaled data back to a DataFrame
df_scaled = pd.DataFrame(df_scaled,index=df_prices.index ,columns=df_prices.columns)

# Displaying the first few rows of the scaled data
df_scaled

,ADN,AEE,AEP,AES,AGR,ALCE,ALE,AMPS,ARIS,ARTNA,...,TVE,UGI,UTL,VGAS,VIA,VST,WEC,WTRG,XEL,YORW
date,,,,,,,,,,,,,,,,,,,,,
2023-03-01,3.222096,-0.354795,0.395483,1.543968,0.593551,0.156775,0.532673,1.496300,2.950102,1.313332,...,-1.349285,2.106191,0.380578,2.034965,3.591652,-1.574295,-0.469367,1.015342,-0.091136,0.530036
2023-03-02,3.160390,-0.054490,0.702810,1.547155,0.797903,0.159748,0.649576,1.510264,3.197922,1.497373,...,-1.673691,2.158398,0.532241,1.568240,3.463654,-1.657741,-0.146614,0.996674,0.193180,0.684372
2023-03-03,3.345507,0.252588,0.910205,1.693698,0.954171,0.162720,0.841418,1.957122,3.088590,1.489371,...,-0.818431,2.223656,0.691751,1.530736,3.256889,-1.633900,0.202848,0.841103,0.395124,0.865717
2023-03-06,3.222096,0.279682,0.962997,1.661841,0.996246,0.168665,0.709529,2.208480,2.702280,1.453364,...,-0.818431,2.117068,0.527011,1.534903,2.961509,-1.043818,0.258493,0.931336,0.381839,0.923592
2023-03-07,3.160390,-0.038685,0.823475,1.505741,0.788888,0.174610,0.358816,2.445873,1.215353,1.445363,...,-0.847921,2.064860,0.469484,1.491564,2.744897,-0.620629,0.042585,0.682422,-0.000791,1.602667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22,-1.492821,-1.863901,-0.259588,-0.058010,-0.754488,-2.934556,1.324052,1.510264,-0.592055,-0.888750,...,2.077954,-0.317458,0.362887,-1.286278,-0.279789,1.880062,-0.772815,-0.400956,0.009062,-0.814315
2023-12-26,-1.519046,-1.933789,-0.191540,0.026994,-0.676399,-4.658568,1.342688,1.845408,-0.478146,-0.850053,...,2.228326,-0.236949,0.547626,-1.298780,-0.210867,1.916836,-0.612747,-0.311777,0.060839,-0.486894
2023-12-27,-1.429265,-1.950095,-0.175989,0.000839,-0.735747,-4.848804,1.233979,1.901265,-0.462958,-0.898933,...,2.769643,-0.234649,0.644014,-1.298780,-0.269943,1.953612,-0.656195,-0.318145,0.008626,-0.467404


In [4]:
df_scaled = df_scaled.T
df_scaled

date,2023-03-01,2023-03-02,2023-03-03,2023-03-06,2023-03-07,2023-03-08,2023-03-09,2023-03-10,2023-03-13,2023-03-14,...,2023-12-15,2023-12-18,2023-12-19,2023-12-20,2023-12-21,2023-12-22,2023-12-26,2023-12-27,2023-12-28,2023-12-29
ADN,3.222096,3.160390,3.345507,3.222096,3.160390,2.944421,2.635894,2.234808,2.080544,2.203955,...,-1.189847,-1.143568,-1.132153,-1.188305,-1.174421,-1.492821,-1.519046,-1.429265,-1.467522,-1.458883
AEE,-0.354795,-0.054490,0.252588,0.279682,-0.038685,0.121626,0.010990,-0.352535,0.157753,0.392093,...,-1.943106,-2.080551,-1.798674,-1.870891,-1.856913,-1.863901,-1.933789,-1.950095,-1.810322,-1.801004
AEP,0.395483,0.702810,0.910205,0.962997,0.823475,0.949802,0.847987,0.616081,0.964882,1.128916,...,0.100089,0.002879,0.080646,-0.271254,-0.325691,-0.259588,-0.191540,-0.175989,-0.074888,-0.096275
AES,1.543968,1.547155,1.693698,1.661841,1.505741,1.572640,1.435654,1.107523,0.992836,1.094780,...,0.023724,-0.084165,0.026994,-0.296674,-0.139744,-0.058010,0.026994,0.000839,0.023724,-0.008970
AGR,0.593551,0.797903,0.954171,0.996246,0.788888,0.960184,0.809925,0.578527,0.791893,0.873033,...,-0.592063,-0.751365,-0.692017,-0.820083,-0.729501,-0.754488,-0.676399,-0.735747,-0.620175,-0.732623
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VST,-1.574295,-1.657741,-1.633900,-1.043818,-0.620629,-0.682220,-0.783547,-1.016003,-1.051766,-0.821296,...,1.767173,1.783425,1.818770,1.796296,1.833071,1.880062,1.916836,1.953612,2.043506,1.982215
WEC,-0.469367,-0.146614,0.202848,0.258493,0.042585,0.127168,0.040361,-0.246777,0.552313,0.834997,...,-0.882579,-0.900872,-0.699642,-1.035788,-0.907732,-0.772815,-0.612747,-0.656195,-0.518991,-0.491552
WTRG,1.015342,0.996674,0.841103,0.931336,0.682422,0.974895,0.769540,0.738429,0.931336,1.080682,...,-0.735377,-0.738561,-0.521985,-0.550649,-0.429620,-0.400956,-0.311777,-0.318145,-0.241707,-0.334071
XEL,-0.091136,0.193180,0.395124,0.381839,-0.000791,0.209125,0.169267,-0.104421,0.448268,0.625048,...,-0.121743,0.088091,0.025412,-0.195321,-0.135369,0.009062,0.060839,0.008626,0.104809,0.110305


In [5]:
tsne = TSNE(n_components=3)
tsne_train = tsne.fit_transform(df_scaled)

# Create a DataFrame for the principal components of the training data
columns = [f'tSne{i+1}' for i in range(tsne_train.shape[1])]
tsne_train_df = pd.DataFrame(tsne_train, index=df_scaled.index, columns=columns)
tsne_train_df


,tSne1,tSne2,tSne3
ADN,-95.438499,46.272820,69.264297
AEE,-17.013498,44.585560,101.140717
AEP,-24.756327,-82.767960,-32.901569
AES,-8.401447,-42.801647,-65.515053
AGR,-89.132523,-24.931795,-19.615118
...,...,...,...
VST,-3.316002,122.695435,68.731514
WEC,9.857067,23.295439,30.599846
WTRG,124.724495,53.203770,18.162798
XEL,35.958939,124.465607,-0.654148


In [6]:
# Apply OPTICS
optics_clustering = OPTICS(min_samples = 2).fit(tsne_train_df)
## make a variable containing the labels for each stock
labels = optics_clustering.labels_
## number of clusters created
number_of_clusters = np.unique(labels)
## Add labels array to the dataframe containing the principle components for the stocks
data_clusters = tsne_train_df
## The feature cluster containing the labels for each stocks is added
data_clusters['cluster'] = labels
    
# Plot the 3D clusters
## Plot the clusters created with OPTICS clustering
fig_clusters = px.scatter_3d(data_clusters, x='tSne1', y='tSne2', z='tSne3',
               color='cluster')
fig_clusters.show()

In [13]:
# Convert the DataFrame to a NumPy array for clustering
X = tsne_train_df.values

# Perform OPTICS clustering
optics = OPTICS(min_samples=2)
labels = optics.fit_predict(X)

# Function to process clusters and calculate pairwise distances
def process_cluster_data(X, labels, stock_symbols):
    unique_labels = np.unique(labels)
    results = []

    for label in unique_labels:
        if label != -1:  # Ignore noise points
            # Extract points in the current cluster
            cluster_points = X[labels == label]
            indices = stock_symbols[labels == label]
            
            # Calculate pairwise distances within the cluster
            distances = pdist(cluster_points)
            distance_matrix = squareform(distances)
            
            # Generate the DataFrame rows for this cluster
            for i in range(len(cluster_points)):
                for j in range(i + 1, len(cluster_points)):
                    results.append({
                        'Pair': indices[i] +'_'+ indices[j],
                        'distance': distance_matrix[i, j]
                    })

    return pd.DataFrame(results)

# Stock symbols as index
stock_symbols = tsne_train_df.index

# Process the clusters and create the DataFrame
df_distances = process_cluster_data(X, labels, stock_symbols)
df_distances

,Pair,distance
0,AMPS_NRGV,41.346001
1,AMPS_SWX,74.499064
2,NRGV_SWX,42.252610
3,CMS_DTE,44.972448
4,CWEN_CWEN-A,42.017479
5,CWEN_WEC,75.432085
6,CWEN-A_WEC,47.884709
7,ALCE_OGS,86.453889
8,ALCE_OPAL,41.983694
9,OGS_OPAL,45.547523


In [26]:
df_cointegrated = pd.read_csv('df_half_life.csv')
df_cointegrated

,coint_t,pvalue,hedge_ratio,hurst_exponent,half_life,crossovers,Pair
0,-3.471468,0.035010,0.159752,0.476953,211.728380,21,ADN_AGR
1,-4.031608,0.006470,0.160480,0.498303,128.708203,9,ADN_BEPC
2,-3.986240,0.007512,0.286938,0.384031,10.659137,17,ADN_CDZI
3,-3.430575,0.039066,-0.311334,0.469681,261.929260,21,ADN_CEG
4,-3.497491,0.032618,-0.462547,0.447678,65.773530,18,ADN_GNE
...,...,...,...,...,...,...,...
242,-3.431832,0.038936,0.718398,0.360882,97.530674,20,UTL_XEL
243,-3.444605,0.037632,0.146167,0.489150,347.047706,8,VIA_WTRG
244,-4.236542,0.003209,1.513684,0.415290,298.155065,34,WEC_WTRG
245,-3.421139,0.040056,1.078228,0.447173,47.516973,23,WEC_YORW


In [27]:
df_cluster = df_distances.merge(df_cointegrated,on='Pair')
df_cluster

,Pair,distance,coint_t,pvalue,hedge_ratio,hurst_exponent,half_life,crossovers
0,AMPS_SWX,74.499064,-4.508942,0.001184,0.289044,0.390763,319.130205,24
1,AES_D,44.417553,-3.869257,0.010932,0.507567,0.361442,265.100514,26
2,ARTNA_NWN,49.960953,-4.130920,0.004631,0.687129,0.349361,49.285167,34
3,NWN_UGI,58.286934,-3.413931,0.040827,1.983993,0.326623,172.094291,18


In [28]:
df_performance = pd.read_csv('metrics_df.csv')
df_performance

,Pair,Annualized Return,Annualized Std Dev,Max Drawdown,Annualized Sharpe Ratio,Omega,Cumulative Return,Best Day,Worst Day
0,ADN_AEE,0.659274,0.214984,-0.073539,3.066611,1.854199,0.181499,0.046066,-0.054038
1,ADN_AEP,0.666317,0.596644,-0.231506,1.116774,5.462545,0.183148,0.101282,-0.225058
2,ADN_AES,-0.146097,0.321846,-0.237292,-0.453934,3.287301,-0.050689,0.070747,-0.051014
3,ADN_AGR,1.301340,0.245524,-0.059885,5.300257,2.569091,0.315903,0.046507,-0.038540
4,ADN_ALCE,-0.379493,0.572920,-0.354438,-0.662385,3.707435,-0.145449,0.166744,-0.181583
...,...,...,...,...,...,...,...,...,...
3316,WEC_XEL,-0.081152,0.075566,-0.066052,-1.073919,1.257997,-0.027491,0.023761,-0.015593
3317,WEC_YORW,-0.047448,0.083599,-0.048106,-0.567569,1.362112,-0.015883,0.014007,-0.016272
3318,WTRG_XEL,-0.002982,0.127491,-0.054190,-0.023391,1.473971,-0.000983,0.044451,-0.016453
3319,WTRG_YORW,-0.063931,0.063991,-0.054818,-0.999055,1.259908,-0.021525,0.010327,-0.010542


In [29]:
df_cluster.merge(df_performance,on='Pair')

,Pair,distance,coint_t,pvalue,hedge_ratio,hurst_exponent,half_life,crossovers,Annualized Return,Annualized Std Dev,Max Drawdown,Annualized Sharpe Ratio,Omega,Cumulative Return,Best Day,Worst Day
0,AMPS_SWX,74.499064,-4.508942,0.001184,0.289044,0.390763,319.130205,24,2.384911,0.286012,-0.049647,8.338499,2.583018,0.494219,0.111601,-0.024956
1,AES_D,44.417553,-3.869257,0.010932,0.507567,0.361442,265.100514,26,-0.049624,0.154581,-0.105985,-0.321022,1.627348,-0.016624,0.032633,-0.037846
2,ARTNA_NWN,49.960953,-4.130920,0.004631,0.687129,0.349361,49.285167,34,0.093354,0.135373,-0.047971,0.689605,1.455371,0.029832,0.034756,-0.045423
3,NWN_UGI,58.286934,-3.413931,0.040827,1.983993,0.326623,172.094291,18,-0.192875,0.103003,-0.068142,-1.872517,1.339657,-0.068142,0.013737,-0.033342


In [37]:
# Compute the pairwise correlations between stocks using their t-SNE features

# Transpose the dataframe to treat stocks as variables for correlation calculation
corr_matrix = df_scaled.T.corr()

# Initialize list to store the results
results = []

# Loop through each element in the correlation matrix to extract pairwise correlations
for i in range(len(corr_matrix.columns)):
    for j in range(i + 1, len(corr_matrix.columns)):
        results.append({
            'Pair': corr_matrix.columns[i]+'_'+corr_matrix.columns[j],
            'correlationship': corr_matrix.iloc[i, j]
        })

# Create DataFrame from results
correlation_stocks_df = pd.DataFrame(results)
correlation_stocks_df = correlation_stocks_df.sort_values(by='correlationship', ascending=False)
correlation_stocks_df

,Pair,correlationship
1551,CWEN_CWEN-A,0.997782
329,AGR_BKH,0.978709
343,AGR_ES,0.977222
1222,BKH_POR,0.975519
2043,ES_WTRG,0.974181
...,...,...
1359,CEG_UGI,-0.902167
1316,CEG_ES,-0.908294
701,ARTNA_CEG,-0.913535
1347,CEG_PNM,-0.920901


In [38]:
df_correlation = correlation_stocks_df.merge(df_cointegrated,on='Pair')
df_correlation

,Pair,correlationship,coint_t,pvalue,hedge_ratio,hurst_exponent,half_life,crossovers
0,AGR_ES,0.977222,-3.787434,0.014088,1.182867,0.371316,6.513701,26
1,BKH_POR,0.975519,-3.470623,0.035090,0.787444,0.230085,36.623478,33
2,ES_WTRG,0.974181,-3.643366,0.021631,0.711377,0.412536,38.710625,26
3,CPK_ORA,0.973617,-4.428511,0.001601,0.820722,0.148780,7.060058,46
4,AGR_WTRG,0.966149,-3.372677,0.045477,0.855295,0.435650,32.041143,30
...,...,...,...,...,...,...,...,...
242,MNTK_POR,-0.786172,-4.749193,0.000463,-0.359962,0.192386,246.370457,25
243,ADN_CEG,-0.792298,-3.430575,0.039066,-0.311334,0.469681,261.929260,21
244,AVA_MNTK,-0.799547,-4.284188,0.002710,-1.201838,0.215330,90.423292,38
245,IDA_MNTK,-0.800064,-4.840909,0.000319,-2.505304,0.158993,332.268299,34


In [41]:
df_correlation.merge(df_performance,on='Pair').head(4)

,Pair,correlationship,coint_t,pvalue,hedge_ratio,hurst_exponent,half_life,crossovers,Annualized Return,Annualized Std Dev,Max Drawdown,Annualized Sharpe Ratio,Omega,Cumulative Return,Best Day,Worst Day
0,AGR_ES,0.977222,-3.787434,0.014088,1.182867,0.371316,6.513701,26,-0.199022,0.084834,-0.075814,-2.346009,1.316523,-0.070486,0.009897,-0.019565
1,BKH_POR,0.975519,-3.470623,0.035090,0.787444,0.230085,36.623478,33,-0.047788,0.049497,-0.023879,-0.965465,1.186399,-0.015999,0.008793,-0.007376
2,ES_WTRG,0.974181,-3.643366,0.021631,0.711377,0.412536,38.710625,26,-0.136136,0.087945,-0.090848,-1.547979,1.345210,-0.047056,0.015080,-0.019769
3,CPK_ORA,0.973617,-4.428511,0.001601,0.820722,0.148780,7.060058,46,0.155662,0.101842,-0.026966,1.528470,1.443658,0.048804,0.018781,-0.020427
